In [1]:
"""experiment
Logic: train model: 1 epoch, training dataset: 4096, eval datasets obtained thru select random subset"""

'experiment\nLogic: train model: 1 epoch, training dataset: 4096, eval datasets obtained thru select random subset'

In [2]:
#get models
%load_ext autoreload
%autoreload 2

In [3]:
from src.data.data import get_in_domain, get_out_domain
from src.data.data import get_random_subsets

in_domain_train, in_domain_test = get_in_domain()
out_domain = get_out_domain()

print(f"In domain:\n{in_domain_train}")
print(in_domain_train[0])

print(f"Out of domain:\n{out_domain}")
print(out_domain[10])

# get_random_subsets(in_domain)

Saving the dataset (0/1 shards):   0%|          | 0/392702 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/9815 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/9832 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/9796 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/9847 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/6692 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/30000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/30000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/30000 [00:00<?, ? examples/s]

In domain:
Dataset({
    features: ['premise', 'hypothesis', 'label', 'idx'],
    num_rows: 261802
})
{'premise': 'you know during the season and i guess at at your level uh you lose them to the next level if if they decide to recall the the parent team the Braves decide to call to recall a guy from triple A then a double A guy goes up to replace him and a single A guy goes up to replace him', 'hypothesis': 'You lose the things to the following level if the people recall.', 'label': 0, 'idx': 1}
Out of domain:
Dataset({
    features: ['premise', 'hypothesis', 'label', 'parse_premise', 'parse_hypothesis', 'binary_parse_premise', 'binary_parse_hypothesis', 'heuristic', 'subcase', 'template'],
    num_rows: 10000
})
{'premise': 'The president avoided the athlete .', 'hypothesis': 'The athlete avoided the president .', 'label': 1, 'parse_premise': '(ROOT (S (NP (DT The) (NN president)) (VP (VBD avoided) (NP (DT the) (NN athlete))) (. .)))', 'parse_hypothesis': '(ROOT (S (NP (DT The) (NN at

In [4]:
from src.model.model import get_model, download_model

# Get student models
student_model_opt125_causal, tokenizer_opt125_causal = get_model(model_name='opt-125m', model_type='CausalLM', pretrained=True)
student_model_opt350_causal, tokenizer_opt350_causal = get_model(model_name='opt-350m', model_type='CausalLM', pretrained=True)

# Get teacher models
teacher_model_opt125_causal, tokenizer_opt125_causal = get_model(model_name='opt-125m', model_type='CausalLM', pretrained=True)
teacher_model_opt350_causal, tokenizer_opt350_causal = get_model(model_name='opt-350m', model_type='CausalLM', pretrained=True)

In [15]:
import json
from src.finetuners.fewshot import batch_fine_tune
from src.data.data import get_random_subsets

# Generate training and evaluation datasets. These should be used for all fine-tuning methods to ensure consistency. np.random should be seeded before this.
train_datasets, eval_dataset_in, eval_dataset_out = get_random_subsets(train_dataset=in_domain_train, 
                                                                       eval_dataset_in=in_domain_test, 
                                                                       eval_dataset_out=out_domain, 
                                                                       train_sample_sizes=[4096],
                                                                       num_trials=10,   # 5
                                                                       eval_sample_size=50)

In [19]:
teacher_model_opt125_causal.device

device(type='cuda', index=0)

In [26]:
from src.finetuners.context_distillation import context_distillation
import pprint

metrics = context_distillation(
    student_model_opt125_causal,
    teacher_model_opt125_causal, 
    tokenizer_opt125_causal, 
    train_datasets[4096][0], 
    num_epochs = 1,
    eval_dataset_in=eval_dataset_in, 
    eval_dataset_out=eval_dataset_out, 
    batch_size=2)
print("Metrics:")
pprint.pprint(metrics)

  0%|          | 0/2048 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

In domain eval metrics:
{'loss': 0.656388390660286, 'accuracy': 0.68, 'runtime': 56.577712297439575, 'samples_per_second': 0.8837402215406074}


  0%|          | 0/7 [00:00<?, ?it/s]

Out of domain eval metrics:
{'loss': 0.8004112792015076, 'accuracy': 0.42, 'runtime': 50.83884787559509, 'samples_per_second': 0.9834998645593269}
Metrics:
{'eval_in_accuracy': 0.68,
 'eval_in_loss': 0.656388390660286,
 'eval_in_runtime': 56.577712297439575,
 'eval_in_samples_per_second': 0.8837402215406074,
 'eval_out_accuracy': 0.42,
 'eval_out_loss': 0.8004112792015076,
 'eval_out_runtime': 50.83884787559509,
 'eval_out_samples_per_second': 0.9834998645593269}
